# Gemini Code Documentation Demo
Use free-tier Gemini to annotate code and produce markdown summaries.

## Setup
- Set `GEMINI_API_KEY` in your environment (free tier works).
- Run cells to configure the client.

In [3]:
import os
import textwrap
import google.generativeai as genai

api_key = os.getenv("GEMINI_API_KEY")
if not api_key:
    raise RuntimeError("Set GEMINI_API_KEY before running the notebook.")

genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-2.5-flash")
print("Model ready")

Model ready


## Document code with inline comments
Send a prompt asking Gemini to add docstrings and inline comments.

In [4]:
code_snippet = textwrap.dedent(
    """
    def compute_total(items):
        total = 0
        for item in items:
            total += item
        return total
    """
)

doc_prompt = (
    "Add concise docstrings and inline comments to the following code without changing its behaviour. "
    "Return only the updated code block.\n\n"
    f"Code:\n{code_snippet}"
)

doc_response = model.generate_content(doc_prompt)
print(doc_response.text.strip())

```python
def compute_total(items):
    """Calculates the sum of all numeric items in an iterable.

    Args:
        items: An iterable (e.g., list, tuple) containing numbers (integers or floats).

    Returns:
        The sum of all numbers in the 'items' iterable.
    """
    total = 0  # Initialize the accumulator for the sum
    for item in items:
        total += item  # Add each item to the running total
    return total
```


## Generate a markdown overview
Use a separate prompt to summarise behaviour, inputs, and edge cases.

In [5]:
summary_prompt = (
    "Create a concise markdown summary of the code below. Include sections for Overview, Key Behaviours, "
    "Inputs/Outputs, and Edge Cases. Use bullet points and keep it brief.\n\n"
    f"Code:\n{code_snippet}"
)

summary_response = model.generate_content(summary_prompt)
print(summary_response.text.strip())

### Overview
*   Calculates the sum of all numeric elements within an iterable.

### Key Behaviours
*   Initializes a `total` accumulator to `0`.
*   Iterates sequentially through each `item` in the input `items` iterable.
*   Adds the value of each `item` to the `total`.
*   Returns the final accumulated `total`.

### Inputs/Outputs
*   **Input:** An iterable (e.g., list, tuple) where elements are expected to be numeric (integers, floats).
*   **Output:** A single numeric value (integer or float) representing the sum of all input items.

### Edge Cases
*   **Empty iterable:** Returns `0`.
*   **Non-numeric items:** Will raise a `TypeError` at runtime during the addition operation (e.g., `0 + "abc"`).
*   **Mixed numeric types (e.g., `int` and `float`):** Performs standard numeric type promotion (e.g., `1 + 2.5` results in `3.5`).
